# Check if representative group knockoffs are working

In [1]:
# load packages for this tutorial
using Revise
using Knockoffs
using LinearAlgebra
using Random
using StatsBase
using Statistics
using ToeplitzMatrices
using Distributions
using Clustering
using ProgressMeter
using LowRankApprox
using Plots
using CSV, DataFrames
gr(fmt=:png);

# some helper functions to compute power and empirical FDR
function TP(correct_groups, signif_groups)
    return length(signif_groups ∩ correct_groups) / max(1, length(correct_groups))
end
function TP(correct_groups, β̂, groups)
    signif_groups = get_signif_groups(β̂, groups)
    return TP(correct_groups, signif_groups)
end
function power(correct_snps, discovered_snps)
    return length(discovered_snps ∩ correct_snps) / length(correct_snps)
end
function FDR(correct_groups, signif_groups)
    FP = length(signif_groups) - length(signif_groups ∩ correct_groups) # number of false positives
    return FP / max(1, length(signif_groups))
end
function FDR(correct_groups, β̂, groups)
    signif_groups = get_signif_groups(β̂, groups)
    return FDR(correct_groups, signif_groups)
end
function get_signif_groups(β, groups)
    correct_groups = Int[]
    for i in findall(!iszero, β)
        g = groups[i]
        g ∈ correct_groups || push!(correct_groups, g)
    end
    return correct_groups
end

┌ Info: Precompiling Knockoffs [878bf26d-0c49-448a-9df5-b057c815d613]
└ @ Base loading.jl:1423


get_signif_groups (generic function with 1 method)

## gnomdAD panel

In [13]:
datadir = "/Users/biona001/Benjamin_Folder/research/4th_project/group_knockoff_test_data"
covfile = CSV.read(joinpath(datadir, "CorG_2_127374341_128034347.txt"), DataFrame)
Σ = covfile |> Matrix{Float64}
Σ = 0.99Σ + 0.01I #ensure PSD

# test on smaller data
idx = 500 # 1261 # includes largest group with 192 members
Σ = Σ[1:idx, 1:idx];

In [3]:
# simulate data
m = 5
p = size(Σ, 1)
k = 10 # number of causal groups
n = 1000 # sample size

# simulate X
μ = zeros(p)
X = rand(MvNormal(μ, Σ), n)' |> Matrix
zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

# define groups
nrep = 2
groups, group_reps = id_partition_groups(X, rep_method=:rss, nrep=nrep, force_contiguous=false)
unique_groups = unique(groups)
countmap(groups) |> values |> collect |> sort

# rep group knockoffs
@time ko = modelX_gaussian_rep_group_knockoffs(X, :maxent, μ, Σ, groups, group_reps, m=m, nrep=nrep)
# @time Xr, Σ11 = modelX_gaussian_rep_group_knockoffs(X, :maxent, μ, Σ, groups, group_reps, m=m, nrep=nrep)
# @time Xr, Xc, X̃r, X̃c, X̃r_correct, X̃c_correct = modelX_gaussian_rep_group_knockoffs(X, :maxent, μ, Σ, groups, group_reps, m=m, nrep=nrep)
# @time Xr, Xc, X̃r, X̃c = modelX_gaussian_rep_group_knockoffs(X, :maxent, μ, Σ, groups, group_reps, m=m, nrep=nrep)

X̃ = ko.X̃
Xr = X[:, group_reps]
Xc = X[:, setdiff(1:p, group_reps)]
X̃r = X̃[:, group_reps]
X̃c = X̃[:, setdiff(1:p, group_reps)];

358 representatives for 1261 variables
 25.929303 seconds (74.35 M allocations: 5.212 GiB, 3.75% gc time, 86.80% compilation time)


In [4]:
group_reps

358-element Vector{Int64}:
    1
    2
    3
    4
    6
    8
   13
   14
   16
   37
   47
   49
   55
    ⋮
 1242
 1244
 1246
 1247
 1249
 1250
 1254
 1255
 1258
 1259
 1260
 1261

In [5]:
cor(X)[1:10, 1:10]

10×10 Matrix{Float64}:
  1.0        -0.0316719   0.389543   …   0.0719384  -0.151748   -0.158195
 -0.0316719   1.0         0.106349      -0.021472   -0.21683    -0.212757
  0.389543    0.106349    1.0            0.123974   -0.441458   -0.442255
 -0.10471    -0.0532544  -0.0952586     -0.0425824   0.0169459   0.0112923
  0.115698    0.512278    0.288368      -0.0328618  -0.397518   -0.394521
  0.114959    0.508248    0.287651   …  -0.0322737  -0.38981    -0.385125
  0.115821    0.509189    0.287436      -0.0278966  -0.390335   -0.385289
  0.0719384  -0.021472    0.123974       1.0        -0.0512828  -0.0439006
 -0.151748   -0.21683    -0.441458      -0.0512828   1.0         0.981216
 -0.158195   -0.212757   -0.442255      -0.0439006   0.981216    1.0

In [6]:
cor(X̃)[1:10, 1:10]

10×10 Matrix{Float64}:
  1.0         -0.00997141   0.352483   …  -0.0622522  -0.0728054
 -0.00997141   1.0          0.127599      -0.164301   -0.0810861
  0.352483     0.127599     1.0           -0.337803   -0.18435
 -0.0758171    0.0165755   -0.0363166     -0.131895   -0.00408338
  0.0677441    0.290962     0.249382      -0.402764   -0.133618
  0.0698103    0.368014     0.141711   …   0.141819   -0.0687323
  0.0362744    0.21004      0.0578114      0.168048   -0.143441
  0.0164684   -0.052292     0.0875766     -0.042192    0.00627809
 -0.0622522   -0.164301    -0.337803       1.0         0.169143
 -0.0728054   -0.0810861   -0.18435        0.169143    1.0

In [7]:
cor(Xr)[1:10, 1:10]

10×10 Matrix{Float64}:
  1.0        -0.0316719    0.389543   -0.10471    …   0.0412699  -0.158815
 -0.0316719   1.0          0.106349   -0.0532544      0.0534632  -0.0421471
  0.389543    0.106349     1.0        -0.0952586      0.0708787  -0.31079
 -0.10471    -0.0532544   -0.0952586   1.0            0.0291335   0.111433
  0.114959    0.508248     0.287651   -0.0561983      0.0349881   0.00652459
  0.0719384  -0.021472     0.123974   -0.0425824  …   0.0183663  -0.0819964
 -0.15403    -0.00711405  -0.0919503   0.186435       0.0282279   0.157422
  0.183439    0.316061     0.363082   -0.0345218     -0.0327824   0.27253
  0.0412699   0.0534632    0.0708787   0.0291335      1.0         0.0138495
 -0.158815   -0.0421471   -0.31079     0.111433       0.0138495   1.0

In [8]:
cor(X̃r)[1:10, 1:10]

10×10 Matrix{Float64}:
  1.0         -0.00997141   0.352483    …   0.00804042  -0.00369549
 -0.00997141   1.0          0.127599        0.051183     0.0714719
  0.352483     0.127599     1.0             0.00266261   0.137882
 -0.0758171    0.0165755   -0.0363166      -0.00883495   0.0926628
  0.0698103    0.368014     0.141711        0.0364606   -0.177994
  0.0164684   -0.052292     0.0875766   …   0.0140068    0.00598495
 -0.108199    -0.02364     -0.035713        0.00832277   0.100231
  0.132174     0.117316     0.105653        0.0429591   -0.118175
  0.00804042   0.051183     0.00266261      1.0         -0.0189152
 -0.00369549   0.0714719    0.137882       -0.0189152    1.0

In [9]:
cor(Xc)[1:10, 1:10]

10×10 Matrix{Float64}:
  1.0          0.983668     -0.397518  …  -0.399443  -0.391371  -0.118392
  0.983668     1.0          -0.390335     -0.392346  -0.383369  -0.123844
 -0.397518    -0.390335      1.0           0.977576   0.978594  -0.22287
 -0.394521    -0.385289      0.981216      0.979198   0.980183  -0.218867
 -0.393931    -0.385144      0.982634      0.981732   0.982777  -0.220375
 -0.00356418  -0.000993417   0.695514  …   0.696372   0.697922   0.230876
 -0.393442    -0.385385      0.981647      0.981923   0.982558  -0.222341
 -0.399443    -0.392346      0.977576      1.0        0.985708  -0.222598
 -0.391371    -0.383369      0.978594      0.985708   1.0       -0.226015
 -0.118392    -0.123844     -0.22287      -0.222598  -0.226015   1.0

In [10]:
cor(X̃c)[1:10, 1:10]

10×10 Matrix{Float64}:
  1.0         -0.00599674  -0.402764  …  -0.147842  -0.173105    0.109841
 -0.00599674   1.0          0.168048     -0.191464   0.0725483  -0.3686
 -0.402764     0.168048     1.0           0.184178   0.154295   -0.227274
 -0.133618    -0.143441     0.169143      0.287329   0.0149983  -0.159129
 -0.121344    -0.153467     0.141233      0.404215   0.245557   -0.0670798
 -0.0150811    0.0152171    0.174025  …   0.111881   0.149561    0.607052
 -0.128207    -0.0330008    0.174625      0.238896   0.344133   -0.172016
 -0.147842    -0.191464     0.184178      1.0        0.229107   -0.116413
 -0.173105     0.0725483    0.154295      0.229107   1.0        -0.0455498
  0.109841    -0.3686      -0.227274     -0.116413  -0.0455498   1.0

**Conclusion**
Seems like $X_r$ and $\tilde{X}_r$ agrees fairly well while $X_c$ and $\tilde{X}_c$ agrees only somewhat. Lets try this in simulations

## One simulation

In [21]:
# simulate data
m = 5
p = size(Σ, 1)
k = 10 # number of causal groups
n = 500 # sample size
μ = zeros(p)

# simulate X
X = rand(MvNormal(μ, Σ), n)' |> Matrix
zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

# define groups
nrep = 5
groups, group_reps = id_partition_groups(X, rep_method=:rss, nrep=nrep, force_contiguous=false)
unique_groups = unique(groups)

# simulate βtrue
βtrue = zeros(p)
βtrue[1:k] .= rand(-1:2:1, k) .* randn(k)
shuffle!(βtrue)
causal_groups = get_signif_groups(βtrue, groups)

# simulate y
ϵ = randn(n)
y = X * βtrue + ϵ

# fully general me
@time me = modelX_gaussian_group_knockoffs(
    X, :maxent, groups, μ, Σ, 
    m = m,
    niter = 10,
    tol = 0.01,    # convergence tolerance
    verbose=false, # whether to print informative intermediate results
)
me_ko_filter = fit_lasso(y, me)
me_power = round(TP(causal_groups, me_ko_filter.βs[3], groups), digits=3)
me_fdr = round(FDR(causal_groups, me_ko_filter.βs[3], groups), digits=3)
me_ssum = sum(abs.(me_ko_filter.ko.S))
@show me_power, me_fdr

# representative ME knockoffs
@time rme = modelX_gaussian_rep_group_knockoffs(
    X, :maxent, μ, Σ, groups, group_reps,
    m = m, 
    nrep = nrep, 
);
rme_ko_filter = fit_lasso(y, rme)
discovered_groups = groups[findall(!iszero, rme_ko_filter.βs[3])] |> unique
rme_power = round(TP(causal_groups, discovered_groups), digits=3)
rme_fdr = round(FDR(causal_groups, discovered_groups), digits=3)
@show rme_power, rme_fdr

 11.531111 seconds (94.64 k allocations: 249.330 MiB, 0.06% gc time)
(me_power, me_fdr) = (0.5, 0.0)
232 representatives for 500 variables
  0.395546 seconds (6.19 k allocations: 249.188 MiB)
(rme_power, rme_fdr) = (0.625, 0.444)


(0.625, 0.444)

### Interpolative decomposition, target FDR = 0.1, nrep=5, m = 5

In [22]:
fdr_hat = 0.0
nsims = 10
for i in 1:nsims
    Random.seed!(i)
    
    # simulate data
    m = 5
    p = size(Σ, 1)
    k = 10 # number of causal groups
    n = 500 # sample size
    μ = zeros(p)

    # simulate X
    X = rand(MvNormal(μ, Σ), n)' |> Matrix
    zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

    # define groups
    nrep = 5
    groups, group_reps = id_partition_groups(X, rep_method=:rss, nrep=nrep, force_contiguous=false)
    unique_groups = unique(groups)

    # simulate βtrue
    βtrue = zeros(p)
    βtrue[1:k] .= rand(-1:2:1, k) .* randn(k)
    shuffle!(βtrue)
    causal_groups = get_signif_groups(βtrue, groups)

    # simulate y
    ϵ = randn(n)
    y = X * βtrue + ϵ

    # fully general me
    @time me = modelX_gaussian_group_knockoffs(
        X, :maxent, groups, μ, Σ, 
        m = m,
        niter = 10,
        tol = 0.01,    # convergence tolerance
        verbose=false, # whether to print informative intermediate results
    )
    me_ko_filter = fit_lasso(y, me)
    me_power = round(TP(causal_groups, me_ko_filter.βs[3], groups), digits=3)
    me_fdr = round(FDR(causal_groups, me_ko_filter.βs[3], groups), digits=3)
    me_ssum = sum(abs.(me_ko_filter.ko.S))
    @show me_power, me_fdr

    # representative ME knockoffs
    @time rme = modelX_gaussian_rep_group_knockoffs(
        X, :maxent, μ, Σ, groups, group_reps,
        m = m, 
        nrep = nrep, 
    );
    rme_ko_filter = fit_lasso(y, rme)
    discovered_groups = groups[findall(!iszero, rme_ko_filter.βs[3])] |> unique
    rme_power = round(TP(causal_groups, discovered_groups), digits=3)
    rme_fdr = round(FDR(causal_groups, discovered_groups), digits=3)
    @show rme_power, rme_fdr
    fdr_hat += rme_fdr
end
fdr_hat /= nsims
println("representative ME knockoff has avg FDR $fdr_hat")

 12.313693 seconds (95.67 k allocations: 249.381 MiB, 0.08% gc time)
(me_power, me_fdr) = (0.429, 0.25)
230 representatives for 500 variables
  0.509597 seconds (6.16 k allocations: 249.041 MiB, 2.71% gc time)
(rme_power, rme_fdr) = (0.714, 0.375)
  7.505731 seconds (60.49 k allocations: 246.737 MiB, 2.55% gc time)
(me_power, me_fdr) = (0.5, 0.0)
244 representatives for 500 variables
  0.471243 seconds (6.89 k allocations: 250.043 MiB, 1.13% gc time)
(rme_power, rme_fdr) = (0.8, 0.333)
 12.215512 seconds (97.08 k allocations: 249.463 MiB, 0.06% gc time)
(me_power, me_fdr) = (0.444, 0.0)
234 representatives for 500 variables
  0.458480 seconds (6.64 k allocations: 249.290 MiB, 0.92% gc time)
(rme_power, rme_fdr) = (0.444, 0.2)
 10.417053 seconds (90.08 k allocations: 248.990 MiB, 0.07% gc time)
(me_power, me_fdr) = (0.333, 0.0)
239 representatives for 500 variables
  0.415355 seconds (6.80 k allocations: 249.696 MiB, 0.92% gc time)
(rme_power, rme_fdr) = (0.778, 0.0)
 10.054782 seconds 

### Increasing nreps to 25

In [23]:
fdr_hat = 0.0
nsims = 10
for i in 1:nsims
    Random.seed!(i)
    
    # simulate data
    m = 5
    p = size(Σ, 1)
    k = 10 # number of causal groups
    n = 500 # sample size
    μ = zeros(p)

    # simulate X
    X = rand(MvNormal(μ, Σ), n)' |> Matrix
    zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

    # define groups
    nrep = 25
    groups, group_reps = id_partition_groups(X, rep_method=:rss, nrep=nrep, force_contiguous=false)
    unique_groups = unique(groups)

    # simulate βtrue
    βtrue = zeros(p)
    βtrue[1:k] .= rand(-1:2:1, k) .* randn(k)
    shuffle!(βtrue)
    causal_groups = get_signif_groups(βtrue, groups)

    # simulate y
    ϵ = randn(n)
    y = X * βtrue + ϵ

    # fully general me
    @time me = modelX_gaussian_group_knockoffs(
        X, :maxent, groups, μ, Σ, 
        m = m,
        niter = 10,
        tol = 0.01,    # convergence tolerance
        verbose=false, # whether to print informative intermediate results
    )
    me_ko_filter = fit_lasso(y, me)
    me_power = round(TP(causal_groups, me_ko_filter.βs[3], groups), digits=3)
    me_fdr = round(FDR(causal_groups, me_ko_filter.βs[3], groups), digits=3)
    me_ssum = sum(abs.(me_ko_filter.ko.S))
    @show me_power, me_fdr

    # representative ME knockoffs
    @time rme = modelX_gaussian_rep_group_knockoffs(
        X, :maxent, μ, Σ, groups, group_reps,
        m = m, 
        nrep = nrep, 
    );
    rme_ko_filter = fit_lasso(y, rme)
    discovered_groups = groups[findall(!iszero, rme_ko_filter.βs[3])] |> unique
    rme_power = round(TP(causal_groups, discovered_groups), digits=3)
    rme_fdr = round(FDR(causal_groups, discovered_groups), digits=3)
    @show rme_power, rme_fdr
    fdr_hat += rme_fdr
end
fdr_hat /= nsims
println("representative ME knockoff has avg FDR $fdr_hat")

 12.011551 seconds (95.67 k allocations: 249.381 MiB, 0.21% gc time)
(me_power, me_fdr) = (0.429, 0.25)
451 representatives for 500 variables
  5.136995 seconds (51.74 k allocations: 266.111 MiB, 0.11% gc time)
(rme_power, rme_fdr) = (0.571, 0.0)
  7.051117 seconds (60.49 k allocations: 246.737 MiB)
(me_power, me_fdr) = (0.5, 0.0)
469 representatives for 500 variables
  7.076789 seconds (64.98 k allocations: 267.282 MiB, 0.07% gc time)
(rme_power, rme_fdr) = (0.7, 0.0)
 11.949892 seconds (97.08 k allocations: 249.463 MiB, 0.06% gc time)
(me_power, me_fdr) = (0.444, 0.0)
444 representatives for 500 variables
  7.579106 seconds (77.50 k allocations: 271.273 MiB, 0.12% gc time)
(rme_power, rme_fdr) = (0.0, 0.0)
 10.252961 seconds (90.08 k allocations: 248.990 MiB)
(me_power, me_fdr) = (0.333, 0.0)
459 representatives for 500 variables
  6.615283 seconds (64.83 k allocations: 265.520 MiB)
(rme_power, rme_fdr) = (0.444, 0.0)
 10.005554 seconds (82.59 k allocations: 248.314 MiB, 0.03% gc tim